# variables

In [1]:
import re
import json
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient

headers = json.loads(r'''{
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate",
    "Accept-Language": "en-US,en;q=0.9",
    "Connection": "keep-alive",
    "Cookie": "trctestcookie=ok; trctestcookie=ok; bblastvisit=1517316262; bblastactivity=0; __utmz=60066354.1517316266.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __gads=ID=0150c2db39515748:T=1517316267:S=ALNI_MaDEROky928259c08-Wiv9vdVzIMA; trc_cookie_storage=taboola%2520global%253Auser-id%3D9fb09c65-d038-47cc-aca3-9186c0df7852-tuct1686a87; __utma=60066354.2080145337.1517316266.1519810697.1519820804.11; __utmc=60066354; __utmt=1; __utmb=60066354.123.0.1519822641336",
    "Host": "www.dogforum.com",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36"
}''')

## Function

In [2]:
def getID():
    infos = soup.select('strong')
    return re.findall('<a href="http://www.dogforum.com/members/(.+)/.+ onclick', str(infos))[0]

In [3]:
def getName():
    infos = soup.select('strong')
    return re.findall("\t(.+)'s Profile", infos[0].text)[0]

In [4]:
def getLevel():
    infos = soup.select('meta[name="description"]')
    return re.findall('<meta content=".+ is a (.+) in the Dog Forum.', str(infos[0]))[0]

In [5]:
def setStatistics():
    for info in infos:
        if(re.findall('Total Posts: (.+)', info.text)):
            totalPosts = int(re.findall('Total Posts: (.+)', info.text)[0].replace(',', ''))
        elif(re.findall('Posts Per Day: (.+)', info.text)):
            postPerDay = float("{0:.2f}".format(float(re.findall('Posts Per Day: (.+)', info.text)[0])))
        elif(re.findall('Join Date: (.+)', info.text)):
            joinDate = re.findall('Join Date: (.+)', info.text)[0]
        elif(re.findall('Referrals: (.+)', info.text)):
            referrals = int(re.findall('Referrals: (.+)', info.text)[0])
        elif(re.findall('Last Activity: (.+)', info.text)):
            lastActivity = re.findall('Last Activity: (.+)', info.text)[0]
            
    try:
        memberDic['TotalPosts'] = totalPosts
        memberDic['PostsPerDay'] = postPerDay
        memberDic['JoinDate'] = joinDate
        memberDic['Referrals'] = referrals
        try:
            memberDic['LastActivity'] = lastActivity
        except:
            memberDic['LastActivity'] = float('NaN')
            print('NO lastActivity field~')
    except:
        print('setStatistics() ERROR!')

In [6]:
def setAbout():
    try:
        about = '<dt class="shade">' + re.findall('<dt class="shade">(.+)</dl>', str(infos))[0]
    except:
        return
    aboutSoup = BeautifulSoup(about, 'lxml')
    item = []
    value = []
    # add data
    for i in aboutSoup.select('dt.shade'):
        item.append(i.text)
    for j in aboutSoup.select('dd'):
        value.append(j.text)
    # add Signature
    item.append('Signature')
    try:
        value.append(soup.select('dl.list_no_decoration > dd#signature')[0].text.strip())
    except:
        return
    
    # iterate it & add it into aboutDic
    for i in range(0, len(item)):
        memberDic[item[i]] = value[i]

In [7]:
def setMemberDic(name, ID, level):
    memberDic['_id'] = ID
    memberDic['name'] = name
    memberDic['level'] = level

## test Function
- use for loop in urls

In [13]:
# url = 'http://www.dogforum.com/members/128001/'
url = 'http://www.dogforum.com/members/165089/'
# url = 'http://www.dogforum.com/members/1/'

In [14]:
resp = requests.get(url, headers=headers)
soup = BeautifulSoup(resp.text, 'lxml')
infos = soup.select('ul.list_no_decoration li')
# all data store in here
memberDic = {}

In [8]:
def getMemberDic(url):
    ID = getID()
    name = getName()
    level = getLevel()
    
    setStatistics()
    setAbout()
    setMemberDic(name, ID, level)
    
    return memberDic

In [16]:
getMemberDic(url)

{'Biography': 'Proud owner of a Pembroke Welsh Corgi. :)',
 'JoinDate': '08-15-2016',
 'LastActivity': '08-15-2016 10:05 PM',
 'Location': 'Los Angeles',
 'PostsPerDay': 0.0,
 'Referrals': 0,
 'Signature': 'Hello',
 'TotalPosts': 1,
 '_id': '165089',
 'level': 'Junior Member',
 'name': 'doggyrich'}

In [24]:
# svalue = ['JoinDate', 'LastActivity', 'PostsPerDay', 'Referrals', 'TotalPosts']

### Function:
- connect to mongoDB

In [10]:
# 把檔案讀進來
urls = []
with open('/home/parker/mydir/crawler/DogForum/memberLink.txt') as inputfile:
    for line in inputfile:
        urls.append(line.strip().split(','))

In [12]:
urls = urls[0]

In [13]:
## to mongoDB
conn = MongoClient('localhost', 27017)
db = conn.dogForum
collection = db.memberInfos

def to_mongoDB(profile):
    results = collection.insert(profile)

## Function
- store data into mongoDB

In [ ]:
for url in urls:
    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.text, 'lxml')
    infos = soup.select('ul.list_no_decoration li')
    # all data store in here
    memberDic = {}
    try:
        post = getMemberDic(url)
        to_mongoDB(post)
    except:
        print(url + '  ERROR!!')

/home/parker/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  import sys
